<a href="https://colab.research.google.com/github/rendalamili/ml-for-table-extraction/blob/main/Camelot_and_Machine_Learning_Debugging_Attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resources

---



https://huggingface.co/transformers/v4.3.3/_modules/transformers/training_args.html#:~:text=[docs]%20@dataclass%20class%20TrainingArguments:%20%22%22%22%20TrainingArguments%20is%20the%20subset%20of

https://github.com/NielsRogge/Transformers-Tutorials/blob/master/Table%20Transformer/Using_Table_Transformer_for_table_detection_and_table_structure_recognition.ipynb

https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html?highlight=cuda#cuda-tensors

https://github.com/microsoft/table-transformer/blob/main/docs/INFERENCE.md

https://camelot-py.readthedocs.io/en/master/

https://github.com/tesseract-ocr/tesseract

https://www.kaggle.com/datasets/sreesankar711/pubtables-subset-100k

https://github.com/microsoft/table-transformer

In [ ]:
# Install packages
!pip install camelot-py[cv] ghostscript PyPDF2==1.26.0 pdf2image transformers matplotlib numpy opencv-python pandas torch tensorflow table-transformer transformers datasets
!apt-get install -y ghostscript
!pip install --upgrade camelot-py[cv]
!pip install pytesseract
!apt-get install -y tesseract-ocr
!apt-get install -y poppler-utils
#!pip install --upgrade pillow datasets
!pip install timm
!kaggle datasets download -d sreesankar711/pubtables-subset-100k
!unzip pubtables-subset-100k.zip -d /content/pubtables



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ghostscript is already the newest version (9.55.0~dfsg1-0ubuntu5.9).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
  Using cached camelot_py-0.11.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached pypdf-5.0.1-py3-none-any.whl.metadata (7.4 kB)
INFO: pip is looking at multiple versions of camelot-py[cv] to determine which version is compatible with other requirements. This could take a while.
  Using cached camelot_py-0.10.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached camelot_py-0.10.0-py3-none-any.whl.metadata (8.3 kB)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the new

In [ ]:
# Import libraries
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# PDF libaries
import camelot
import ghostscript
import PyPDF2 as pypdf
from pdf2image import convert_from_path

# Image processsing libaries
import pytesseract
import cv2
import PIL
from PIL import Image
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torchvision.transforms as transforms
from PyPDF2 import PdfFileReader

# Pytorch libraries
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Transformer and datasets
import transformers
from transformers import TableTransformerForObjectDetection, Trainer, TrainingArguments, pipeline
from transformers import AutoImageProcessor, TableTransformerForObjectDetection, TrainingArguments, Trainer
from transformers import DefaultDataCollator
from transformers import Trainer, TrainingArguments, default_data_collator
import datasets
from datasets import load_dataset
from datasets import Dataset, Features, ClassLabel, Value, Image as ImageType

# Google Collab
from google.colab import files

In [ ]:
# Upload PDF file
upload = files.upload()
pdf_file = list(upload.keys())[0]

Saving PMC1064095_table_1.pdf to PMC1064095_table_1.pdf


In [ ]:
def extract_pdf_table(pdf_file):
    # Initialise tables list to store extracted tables
    tables = []

    # Extract tables using Camelot (stream)
    try:
        tables = camelot.read_pdf(pdf_file, pages='all', flavor='stream')
    except Exception as e:
        print(f"Error extracting table with Camelot (stream): [{e}]")
        # Extract using Camelot (lattice)
        try:
            tables = camelot.read_pdf(pdf_file, pages='all', flavor='lattice')
        except Exception as e:
            print(f"Error extracting table with Camelot (lattice): [{e}]")

    # If tables were sucessfully extracted using Camelot, return them
    if tables:
        print(f"Extracted {len(tables)} tables using Camelot.")
        return [table.df.values.tolist() for table in tables]

    # If Camelot was unsucessful use OCR
    print("Using OCR for image-based table extraction.")

    # Convert PDF to image using pdf2image
    images = convert_from_path(pdf_file, poppler_path="/usr/bin", fmt='png')

    # Initialise list to store extracted text from images
    extract_text_table = []

    # Extract text from images using Tesseract
    for i, image in enumerate(images):
      image = image = image.convert('RGB')
      text = pytesseract.image_to_string(image)
      extract_text_table.append(text)
      print(f"Extracted text from image [{i + 1}/{len(images)}]")

    # Process extracted text into structured data
    extracted_data = []
    for text in extract_text_table:
        rows = text.split('\n')  # Split text into rows
        for row in rows:
            if row.strip():  # Ignore empty rows
                extracted_data.append(row.strip().split())  # Split into columns

    return extracted_data  # Return structured data

    #upload = files.upload()
    #pdf_file = list(upload.keys())[0]

# Example use of extract_pdf_table function
if __name__ == "__main__":
    data = extract_pdf_table(pdf_file)
    print(data)

Using OCR for image-based table extraction.
Extracted text from image [1/1]
[['Odds', 'ratios', 'for', 'breast', 'cancer', 'and', 'total', 'PCBs', 'in', 'relation', 'to', 'CYP1A1', 'M2', 'genotypes', 'in', 'white', 'women'], ['Total', 'PCBs*', 'CYPTAT', 'M2', 'genotype', 'All', 'partiapants', 'Premenopausal', 'Postmenopausal'], ['Patienta/controls', '=', 'OR(95%', 'Cl!', 'Patienta/controls', 'OR', '(85%', 'Cir', 'Patients/controla', 'OR', '(95%', 'Cijt'], ['<0,949', 'Non-M2', '210/158', 'Referent', '141/98', 'Referent', '69/60', 'Referent'], ['20,949', 'NoaM2', '138/166', '0.7', '(0,8~1.0)', '45/51', '0,6', '(04~1.0)', '93/115', '0.8', '(0.51.2)'], ['<0.949', 'Any', 'M2', '11/20', '0.4', '(0.2-0.8)', '816', '0.3', '(0.1-0.8)', 'a4', '0.7', '(0.1=3.1)'], ['20.349', 'Any', 'M2', '15/14', '0.9', '(0.4-1,9)', '62', '2.1', '(0,.4-10.6)', '912', '0.7', '(0,3-1,9)'], ['ICR', '(95%', 'Cl)', '0.8', '(0.1,', '1.6)', '2.1', '(-1.3,5.5)', '0.3', '(-0.9,1.5)'], ['Seiad', '|', 'total', 'sepees', 'PC

In [ ]:
# Convert table to JSON
def json_table(table):
  # Check if table has DataFrame 'df' attribute
  if hasattr(table, 'df'):
    header = table.df.values.tolist() # Get df header
    rows = table.df.values.tolist() # Get df rows
    # Return list of dictionaries
    return [dict(zip(header, row)) for row in rows]
  else:
    # If table is a list or tuple
    if table and isinstance(table, (list, tuple)) and len(table) > 1:
      header = table[0] # First element is header
      rows = table[1:] # Remaining elements are rows
      # Return list of dictionaries
      return [dict(zip(header, row)) for row in rows]
    else:
      print("Invalid table format for JSON conversion.")
      return []


In [ ]:
# Extract table from PDF file and store as tables
tables = extract_pdf_table(pdf_file)

# Convert extracted table to JSON
json_tables = [json_table(table) for table in tables]
with open('tables.json', 'w') as f:
  json.dump(json_tables, f)

Using OCR for image-based table extraction.
Extracted text from image [1/1]


In [ ]:
# Convert extracted table to an image
for i, table in enumerate(tables):
  df = pd.DataFrame(table)

  # Create new figure and axis for plotting
  fig, ax = plt.subplots()
  ax.axis('off')

  # Create table plot with DataFrame values and column labels
  plot_table = ax.table(cellText=df.values, colLabels=df.columns, loc='center')

  # Save plot as PNG image
  plt.savefig(f'table_{i}.png', bbox_inches='tight', pad_inches=0)
  plt.close()

In [ ]:
# Load pre-trained Table Transformer model
model = TableTransformerForObjectDetection.from_pretrained('microsoft/table-transformer-detection')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-detection were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Load image processor
image_processor = AutoImageProcessor.from_pretrained("microsoft/table-transformer-detection")

In [ ]:
def load_dataset_with_labels(images_dir, annotations_dir, limit=100):
    dataset = {}

    for filename in os.listdir(annotations_dir):
        if filename.endswith('_words.json'):
            with open(os.path.join(annotations_dir, filename)) as f:
                annotations = json.load(f)

            image_name = filename.replace('_words.json', '.jpg')
            image_path = os.path.join(images_dir, image_name)

            if os.path.exists(image_path):
                if image_path not in dataset:
                    dataset[image_path] = []

                # Collect annotations
                for item in annotations:
                    labels = {
                        'text': item['text'],
                        'bbox': item['bbox']
                    }
                    dataset[image_path].append(labels)

                if len(dataset) >= limit:
                    break
            else:
                print(f"Image not found: {image_path}")

    final_dataset = [{'image_path': img_path, 'labels': labels} for img_path, labels in dataset.items()]
    return final_dataset[:limit]

# Define directories
images_dir = '/content/pubtables/subset/img_train'
annotations_dir = '/content/pubtables/subset/words_train'

# Load the dataset with a limit of 100 unique images
train_dataset = load_dataset_with_labels(images_dir, annotations_dir, limit=100)

# Print the first 5 items to check
print(train_dataset[:5])

# Test dataset
print(train_dataset[0:1])

# Subset of 100 examples from the training dataset
subset_training_dataset = train_dataset[:100]

# Check
for img_path in subset_training_dataset:
    print(img_path)


"""Fix the image transformation by ensuring conversion to torch.Tensor during preprocessing."""
def preprocess_function(examples):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to 224x224
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    processed_data = []

    for example in examples:
        image_column = 'image_path' if 'image_path' in example else 'image'

        try:
            # Process the image
            pil_image = Image.open(example[image_column]).convert("RGB")
            tensor_image = transform(pil_image)

            # Process the labels
            class_labels = []
            boxes = []

            for item in example['labels']:
                # Assuming each item has a 'bbox' and 'text'
                class_labels.append(0)
                boxes.append(item['bbox'])
            processed_data.append({
                "pixel_values": tensor_image,
                "labels": {
                    "class_labels": torch.tensor(class_labels),
                    "boxes": torch.tensor(boxes)
                }
            })

        except Exception as e:
            print(f"Error processing image: {e}")
            processed_data.append({
                "pixel_values": torch.zeros(3, 224, 224),  # Placeholder
                "labels": {
                    "class_labels": torch.tensor([0]),
                    "boxes": torch.tensor([[0, 0, 0, 0]])
                }
            })

    return processed_data


[{'image_path': '/content/pubtables/subset/img_train/PMC1636628_table_1.jpg', 'labels': [{'text': 'Table', 'bbox': [32.358585858585855, 1.8062752525252108, 59.18428030303029, 13.484065656565576]}, {'text': '2:', 'bbox': [61.804974747474745, 1.8062752525252108, 70.11344696969695, 13.484065656565576]}, {'text': 'summary', 'bbox': [72.78150252525252, 1.8062752525252108, 117.34356060606058, 13.484065656565576]}, {'text': 'of', 'bbox': [120.01161616161616, 1.8062752525252108, 129.05560606060604, 13.484065656565576]}, {'text': 'the', 'bbox': [131.72366161616162, 1.8062752525252108, 147.28266414141413, 13.484065656565576]}, {'text': 'JOA', 'bbox': [149.97891414141412, 1.8062752525252108, 170.05183080808078, 13.484065656565576]}, {'text': 'scoring', 'bbox': [172.71988636363636, 1.8062752525252108, 206.67222222222222, 13.484065656565576]}, {'text': 'system', 'bbox': [209.3372601010101, 1.8062752525252108, 242.45122474747473, 13.484065656565576]}, {'text': 'for', 'bbox': [245.1162752525252, 1.80

In [ ]:
"""When an erroneous sample is encountered during collation, None should be returned instead of batch/{}. The Trainer can then ignore this sample.
"""
def custom_data_collator(features):
    try:
        # Initialise a dictionary to hold batch data
        batch = {}

        # Stack pixel values into tensor
        batch['pixel_values'] = torch.stack([f['pixel_values'] for f in features])

        # Handle labels if they exist
        if 'labels' in features[0]:
            batch['labels'] = [f['labels'] for f in features]

        return batch

    except Exception as e:
        print(f"Error during collation: {e}")
        return None  # Return None to indicate an erroneous sample



In [ ]:
"""Replace the raw dataset with the preprocessed subset."""

# Preprocess the dataset
processed_dataset = preprocess_function(subset_training_dataset)

# Replace the training dataset with the pre-processed subset
train_dataset = [{"pixel_values": item["pixel_values"], "labels": item["labels"]} for item in processed_dataset]

# Print to confirm the structure of train_dataset
print(train_dataset[:5])



[{'pixel_values': tensor([[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 0.9843, 0.9451],
         [1.0000, 1.0000, 1.0000,  ..., 0.9922, 0.9686, 0.9059],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 0.9529, 0.8431],
         ...,
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 0.9765, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

        [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 0.9843, 0.9451],
         [1.0000, 1.0000, 1.0000,  ..., 0.9922, 0.9686, 0.9059],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 0.9529, 0.8431],
         ...,
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 0.9765, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

        [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 0.9843, 0.9451],
         [1.0000, 1.0000, 1.0000,  ..., 0.9922, 0.9686, 0.9059],
         [1.0000, 1.0000, 1.0000,  ..., 

In [ ]:
# Convert the processed dataset into a format suitable for the Trainer
train_dataset = [{"pixel_values": item["pixel_values"], "labels": item["labels"]} for item in processed_dataset]

In [ ]:
from transformers import TrainingArguments

training_arguments = TrainingArguments(
    output_dir='./results',                    # Model prediction directory
    evaluation_strategy='epoch',               # Evaluation strategy during training
    save_strategy='epoch',                     # Save strategy during training
    save_total_limit=2,                        # Maximum number of checkpoints to save
    save_steps=100,                            # Save every 100 steps
    learning_rate=2e-5,                        # Learning rate
    per_device_train_batch_size=16,            # Training batch size
    per_device_eval_batch_size=16,             # Evaluation batch size
    num_train_epochs=3,                        # Training epochs
    logging_dir='./logs',                      # Logging directory
    logging_steps=10,                          # Log training
    remove_unused_columns=False,               # Do not remove unused columns in dataset
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

# Ensure your train_dataset is in the correct format
trainer = Trainer(
    model=model,                               # Trained model
    args=training_arguments,                   # Training arguments
    train_dataset=train_dataset,               # Training dataset (use processed dataset)
    data_collator=custom_data_collator         # Data collator for batch
)


In [ ]:
# Define directories for validation data
images_val_dir = '/content/pubtables/subset/img_val'
annotations_val_dir = '/content/pubtables/subset/words_val'

# Load the validation dataset with a limit of 100 unique images (adjust as needed)
val_dataset = load_dataset_with_labels(images_val_dir, annotations_val_dir, limit=100)

# Preprocess the validation dataset
processed_val_dataset = preprocess_function(val_dataset)

# Convert the processed dataset into a format suitable for the Trainer
eval_dataset = [{"pixel_values": item["pixel_values"], "labels": item["labels"]} for item in processed_val_dataset]

# Set the validation dataset in the Trainer
trainer.eval_dataset = eval_dataset


In [ ]:
# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,155.289505
2,142.755300,155.242126
3,164.106800,155.231430


TrainOutput(global_step=21, training_loss=149.58989061628068, metrics={'train_runtime': 217.2672, 'train_samples_per_second': 1.381, 'train_steps_per_second': 0.097, 'total_flos': 7802176899686400.0, 'train_loss': 149.58989061628068, 'epoch': 3.0})